In [1]:

import sys, os, importlib

# This is the repo root that contains the "musicbert_hf" folder
project_root = "/rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf"

# Make sure it's first on sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Drop already-loaded venv version of musicbert_hf, if any
if "musicbert_hf" in sys.modules:
    del sys.modules["musicbert_hf"]

import musicbert_hf
for name in list(sys.modules.keys()):
    if name.startswith("musicbert_hf"):
        del sys.modules[name]

import musicbert_hf
from musicbert_hf.data import HDF5Dataset
import inspect

print("Now using:", musicbert_hf.__file__)
print("HDF5Dataset from:", inspect.getsourcefile(HDF5Dataset))
print("Now using:", musicbert_hf.__file__)
print("Package directory:", os.path.dirname(musicbert_hf.__file__))
import sys
import os
from functools import partial
from transformers import Trainer, TrainingArguments
import os

#sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from musicbert_hf.data import HDF5Dataset, collate_for_musicbert_fn
from musicbert_hf.metrics import compute_metrics, compute_metrics_multitask
from helpers import set_seed, load_baseline_params, get_dataset, create_hyperparams_dict, load_model
from config import load_config
from musicbert_hf.models import freeze_layers, MusicBertTokenClassification, MusicBertMultiTaskTokenClassification, MusicBertMultiTaskTokenClassConditioned


/home/ui556004/projects/musicbert_hf/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now using: /rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf/musicbert_hf/__init__.py
HDF5Dataset from: /rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf/musicbert_hf/data.py
Now using: /rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf/musicbert_hf/__init__.py
Package directory: /rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf/musicbert_hf


In [16]:
import sys, os

project_root = "/rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Remove *all* musicbert_hf-related modules
for name in list(sys.modules.keys()):
    if name.startswith("musicbert_hf"):
        del sys.modules[name]

import musicbert_hf
from musicbert_hf.data import HDF5Dataset
import inspect

print("Now using:", musicbert_hf.__file__)
print("HDF5Dataset from:", inspect.getsourcefile(HDF5Dataset))

Now using: /rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf/musicbert_hf/__init__.py
HDF5Dataset from: /rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf/musicbert_hf/data.py


In [25]:
import musicbert_hf
import inspect
from musicbert_hf.data import HDF5Dataset

print("MusicBERT package:", musicbert_hf.__file__)
print("HDF5Dataset defined in:", inspect.getsourcefile(HDF5Dataset))

MusicBERT package: /rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf/musicbert_hf/__init__.py
HDF5Dataset defined in: /rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf/.venv/src/musicbert-hf/musicbert_hf/data.py


In [11]:
args =load_config("test_params.yaml")

model, config = load_model(args)
model.config.targets = list(config.targets)

test_args = TrainingArguments(
#output_dir=best_model_dir,
per_device_eval_batch_size=args.batch_size,
report_to=None,
do_train=False,
do_eval=True,
)



evaluating the model from hpo
Head for quality
RobertaSequenceTaggingHead(
  (input_dropout): Dropout(p=0, inplace=False)
  (hidden_layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=768, out_features=732, bias=True)
      (1): Tanh()
      (2): Dropout(p=0.09983689107917987, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=732, out_features=571, bias=True)
      (1): LayerNorm((571,), eps=1e-05, elementwise_affine=True)
      (2): ReLU()
      (3): Dropout(p=0.2571172192068058, inplace=False)
    )
    (2): Sequential(
      (0): Linear(in_features=571, out_features=473, bias=True)
      (1): Identity()
      (2): Tanh()
      (3): Dropout(p=0.29620728443102123, inplace=False)
    )
  )
  (out_proj): Linear(in_features=473, out_features=15, bias=True)
)
Head for inversion
RobertaSequenceTaggingHead(
  (input_dropout): Dropout(p=0, inplace=False)
  (hidden_layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=768, out_featur

In [12]:
import torch
def _get_dataset(config, split):
    data_dir = getattr(config, f"{split}_dir")
    device = torch.device("cpu")
    print(f"loading to device {device}")
    dataset = HDF5Dataset(
        os.path.join(data_dir, "events.h5"),
        config.target_paths(split),
        conditioning_path=config.conditioning_path(split),
        device = device
    )
    return dataset
compute_metrics_fn = partial(
compute_metrics_multitask, task_names=args.targets
) if args.multitask else compute_metrics

test_dataset = _get_dataset(args, "test")
test_trainer = Trainer(
    model=model,
    args=test_args,
    data_collator=partial(collate_for_musicbert_fn, multitask=args.multitask),
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_fn,
)


loading to device cpu


In [17]:
study.trials

[FrozenTrial(number=0, state=1, values=[0.8286995266022146], datetime_start=datetime.datetime(2026, 1, 4, 14, 27, 24, 797362), datetime_complete=datetime.datetime(2026, 1, 4, 14, 47, 34, 78001), params={'num_linear_layers_quality': 3, 'layer_dim_quality_0': 732, 'layer_dim_quality_1': 571, 'layer_dim_quality_2': 473, 'layer_dim_quality_3': 146, 'layer_dim_quality_4': 146, 'layer_dim_quality_5': 74, 'activation_fn_quality_0': 'tanh', 'activation_fn_quality_1': 'relu', 'activation_fn_quality_2': 'tanh', 'activation_fn_quality_3': 'relu', 'activation_fn_quality_4': 'relu', 'activation_fn_quality_5': 'gelu', 'input_dropout_quality': 0.3925879806965068, 'pooler_dropout_quality_0': 0.09983689107917987, 'pooler_dropout_quality_1': 0.2571172192068058, 'pooler_dropout_quality_2': 0.29620728443102123, 'pooler_dropout_quality_3': 0.023225206359998862, 'pooler_dropout_quality_4': 0.3037724259507192, 'pooler_dropout_quality_5': 0.08526206184364576, 'normalisation_quality_0': 'layer', 'normalisation

In [41]:
print(f" using model with the args {args}")
print("Evaluating best model on test set...")
test_results = test_trainer.evaluate()
for k, v in test_results.items():
    print(f"{k}: {v:.4f}")

 using model with the args Config(data_dir='/work/ui556004/data/rnbert/datasets/rnbert_seqs/data/', checkpoint_path='/hpcwork/ui556004/musicbert_hf/results/baseline_finetune_3/checkpoint-47000', targets=['quality', 'inversion', 'key_pc_mode', 'primary_alteration_primary_degree_secondary_alteration_secondary_degree'], output_dir_base='results_baseline', conditioning=None, log_dir='/home/ui556004/tmp/musicbert_hf_logs', num_epochs=0, batch_size=4, learning_rate=0.00025, warmup_steps=0, max_steps=-1, hf_repository=None, hf_token=None, DEBUG=True, RUN_NAS=False, num_trials=1, wandb_project=None, wandb_name=None, freeze_layers=None, job_id='1767522715', seed=42, name=None, optuna_storage=None, optuna_name=None, sampler_path=None, time_limit=None, baseline=True, trial_number=None, data_dir_for_metadata=None, msdebug=False, overwrite=False, compound_token_ratio=8, ignore_specials=4, task='musicbert_multitask_sequence_tagging', head='sequence_multitask_tagging_head', max_examples=None, dataset

/home/ui556004/projects/musicbert_hf/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ui556004/projects/musicbert_hf/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: polinaitsme (polinaitsme-RWTH Aachen University) to https:

eval_loss: 0.6170
eval_model_preparation_time: 0.0023
eval_quality_precision: 0.6606
eval_quality_recall: 0.6173
eval_quality_accuracy: 0.8653
eval_inversion_precision: 0.8657
eval_inversion_recall: 0.8464
eval_inversion_accuracy: 0.8730
eval_key_pc_mode_precision: 0.7935
eval_key_pc_mode_recall: 0.8332
eval_key_pc_mode_accuracy: 0.8152
eval_primary_alteration_primary_degree_secondary_alteration_secondary_degree_precision: 0.1979
eval_primary_alteration_primary_degree_secondary_alteration_secondary_degree_recall: 0.1710
eval_primary_alteration_primary_degree_secondary_alteration_secondary_degree_accuracy: 0.7557
eval_precision: 0.6294
eval_recall: 0.6170
eval_accuracy: 0.8273
eval_runtime: 1404.6335
eval_samples_per_second: 0.8410
eval_steps_per_second: 0.2110


In [13]:
print(f" using model with the args {args}")
print("Evaluating best model on test set...")
test_results = test_trainer.evaluate()
for k, v in test_results.items():
    print(f"{k}: {v:.4f}")

 using model with the args Config(data_dir='/work/ui556004/data/rnbert/datasets/rnbert_seqs/data/', checkpoint_path='/hpcwork/ui556004/results/musicbert_hf/checkpoints/new_nas/trial_0000/checkpoint-33000', targets=['quality', 'inversion', 'key_pc_mode', 'primary_alteration_primary_degree_secondary_alteration_secondary_degree'], output_dir_base='0results_hpo', conditioning=None, log_dir='/home/ui556004/tmp/musicbert_hf_logs', num_epochs=0, batch_size=4, learning_rate=0.00025, warmup_steps=0, max_steps=-1, hf_repository=None, hf_token=None, DEBUG=True, RUN_NAS=False, num_trials=1, wandb_project=None, wandb_name=None, freeze_layers=None, job_id='1767629219', seed=42, name=None, optuna_storage='sqlite:///optuna_nas.db', optuna_name='new_nas', sampler_path=None, time_limit=None, baseline=False, trial_number=0, data_dir_for_metadata=None, msdebug=False, overwrite=False, compound_token_ratio=8, ignore_specials=4, task='musicbert_multitask_sequence_tagging', head='sequence_multitask_tagging_he

KeyboardInterrupt: 

In [ ]:
print(compute_metrics_fn)

functools.partial(<function compute_metrics_multitask at 0x15370e9e4c20>, task_names=['quality', 'inversion', 'key_pc_mode', 'primary_alteration_primary_degree_secondary_alteration_secondary_degree'])


In [15]:
from functools import partial

if isinstance(compute_metrics_fn, partial):
    real_fn = compute_metrics_fn.func
else:
    real_fn = compute_metrics_fn

print(real_fn)

<function compute_metrics_multitask at 0x15370e9e4c20>


In [16]:
import inspect

print("Defined in:", inspect.getsourcefile(real_fn))

Defined in: /rwthfs/rz/cluster/home/ui556004/projects/musicbert_hf/musicbert_hf/metrics.py
